<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откроем-и-изучим-файл" data-toc-modified-id="Откроем-и-изучим-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откроем и изучим файл</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Разбиение-данных-на-выборки" data-toc-modified-id="Разбиение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбиение данных на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистиечская-регрессия" data-toc-modified-id="Логистиечская-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистиечская регрессия</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Проверка-моделей-на-адекватность" data-toc-modified-id="Проверка-моделей-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка моделей на адекватность</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Общий-вывод:" data-toc-modified-id="Общий-вывод:-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод:</a></span></li></ul></div>

# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов компании "Мегалайн", которые перешли на тарифы "Смарт" и "Ультра".

*Цель работы* - построить модель для задачи классификации, которая выберет подходящий тариф для новых клиентов. 

*План действий:*
- откроем и изучим файл с данными;
- разделим выборки;
- исследуем модели;
- с помощью тестовой выборки выберем модель с максимально большим значением *accuracy*;
- сформулируем общий вывод.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from random import randint
from sklearn.metrics import accuracy_score
import numpy as np


df = pd.read_csv('/datasets/users_behavior.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод:

Входные данные представляют собой таблицу из 3214 строк, каждая из которых описывает одного клиента компании"Мегалайн" по 5 признакам.

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Также были добавлены библиотеки и функции, которые нужны для выполнения проекта.

В техническом описании сказано, что предобработка данных уже осуществленна, а значит сразу можно приступить к обучению моделей.

## Разбиение данных на выборки

Для того чтобы обучать модели нам потребуется разбить входной датасет на 3 части: тренировочную, валидационную и тестовую выборки.

In [2]:
# методом train_test_split сначала отделим тренировочную выборку в размере 60% от всех данных
train, other = train_test_split(df, test_size=0.4, random_state=5)
# затем оставшуюся часть поделим поровну между валидационной и тестовой выборками.
valid, test = train_test_split(other, test_size=0.5, random_state=5)

Целевым признаком является столбец *is_ultra*. Отделим его от остальных признаков для каждой выборки.

In [3]:
train_features = train.drop('is_ultra', axis=1)
train_target = train['is_ultra']

valid_features = valid.drop('is_ultra', axis=1)
valid_target = valid['is_ultra']

test_features = test.drop('is_ultra', axis=1)
test_target = test['is_ultra']

## Исследование моделей

В целях поиска лучшего результата, попробуем несколько моделей машинного обучения.

### Дерево решений
Первой попробуем модель дерева решений. Так как данная модель склонна к переобучению при большой глубине дерева и недообучения при малой - в цикле переберем максимальную глубину в диапазоне от 1 до 10. Выведем на экран лучший показатель и соответсвующую глубину дерева решений.

In [4]:
best_score = 0
for depth in range(1, 11):
    # помещаем в model_tree алгоритм дерева решений
    model_tree = DecisionTreeClassifier(random_state=5, max_depth=depth)
    # обучаем на тренировочной выборке
    model_tree.fit(train_features, train_target)
    # смотрим на правильность предсказаний
    score = model_tree.score(valid_features, valid_target)
    # и наконец выявляем лучшую глубину
    if score > best_score:
        best_score = score
        best_depth = depth
print(f'Лучший результат = {best_score}, глубина = {best_depth}.')

Лучший результат = 0.8009331259720062, глубина = 6.


### Случайный лес
Следующей опробуем модель случаного леса. Так как эта можель тоже склонна к переобучению, ведь лес по сути состоит из решающи деревьев, то также будем перебирать максимальную глубину в поисках лучшей. Помимо этого стоит перебрать разные количества деревьев, рассмотрим диапазон до 50 деревьев с шагом в 5.

In [5]:
best_score = 0
for est in range(5, 50, 5):
    for depth in range(1, 10):
        model_forest = RandomForestClassifier(random_state=5, max_depth=depth, n_estimators=est)
        model_forest.fit(train_features, train_target)
        score = model_forest.score(valid_features, valid_target)
        if score > best_score:
            best_score = score
            best_depth = depth
            best_est = est
print(f'Лучший результат = {best_score}, количество деревьев = {best_est}, глубина = {best_depth}.')

Лучший результат = 0.8102643856920684, количесвто деревьев = 10, глубина = 5.


Случайный лес показал резульат лучше на 1%.

### Логистиечская регрессия

Осталось проверить последнюю модель. 

In [6]:
# записываем в переменную model_logreg алгоритм логистической регрессии
# указываем параметр solver='liblinear', чтобы избежать предупреждения о будущей ошибке
model_logreg = LogisticRegression(random_state=5, solver='liblinear')
model_logreg.fit(train_features, train_target)
model_logreg.score(valid_features, valid_target)

0.7589424572317263

### Вывод:

По тестам на валидационной выборке лучшего всего себя показала модель **случаного леса** (≈81% точности). Немного хуже справилась модель решающего дерева (≈80% точности). И хуже всех - модель логистической регрессии с точностью в райное 76%.

## Проверка моделей на тестовой выборке

Теперь проверим модели с помощью тестовой выборки и окончательно выберем лучшую.

In [7]:
print('Дерево решений          - ',model_tree.score(test_features, test_target))
print('Случайный лес           - ', model_forest.score(test_features, test_target))
print('Логистическая регрессия - ', model_logreg.score(test_features, test_target))

Дерево решений          -  0.776049766718507
Случайный лес           -  0.7978227060653188
Логистическая регрессия -  0.7371695178849145


### Вывод:
Наилучшей моделью остается модель созданная алгоритмом случайного леса, правильность предсказаний упала всего на 1,2%. У остальных моделей точность упала на 2-3%, что говорит о том, что они немного переучились.

## Проверка моделей на адекватность

Сравним наши показатели с моделью, которая случайно предсказывает ответ.

In [8]:
# с помощью функции randint создадим список равный по длине test_target, в котором будут случайно указаны ответы
rndm = [randint(0, 1) for _ in range(len(test_target))]
# так как ответы каждый раз могут сильно различаться , возьмем средние из ста таких экспериментов
rndm_score = np.mean([accuracy_score(rndm, test_target) for _ in range(100)])
rndm_score

0.5178849144634526

### Вывод:
Слуйчайная модель предсказывает правильные ответы с вероятность около 50%. Из этого следует, что любая наша модель справляется лучше мининмум на 23%.

## Общий вывод:

Результат исследования показал, что аналитикам компании "Мегалайн" лучше воспользоваться **моделью случайного леса с параметрами: количество деревьев = 10, глубина деревьев = 5.** Такая модель лучше всего выберет какому клиенту какой тариф лучше предложить.